In [1]:
#The following is code to estimate capital depreciaion and adjustment costs in the 2 country, 2 sector model in EKNR (2016)
import os
os.getcwd()
import numpy as np
import pandas as pd

In [2]:
#Load the data
k_raw = pd.read_excel("Data/BEA_tab_11_k_stock.xls")
k_raw.head()
k_raw = k_raw.drop([0,1,2,3])
k_raw.head()
k_raw.columns = k_raw.loc[4,:]
k_raw = k_raw.drop([4, 5])

#keep only fixed assets and consumer durable goods, fixed assets, private, government, and consumer durbales
#index: 6, 7, 8, 14, 20
list_to_keep = [6, 7, 8, 14, 20]
k_raw = k_raw.loc[list_to_keep]
# k_raw = k_raw.drop(['Line'], axis=1)
k_raw =k_raw.reset_index()
k_raw = k_raw.drop(['index'], axis=1)
k_raw.rename(columns={np.nan:'type'}, inplace=True)

#reformat and create changes, which is (t+1)/(t)
k_raw = k_raw.T
k_raw.columns = k_raw.loc['type',:]
k_raw = k_raw.drop(['Line', 'type'], axis = 0)
k_data_hat =k_raw.div(k_raw.shift(1))


In [3]:
#Load the data
inv_raw = pd.read_excel("Data/BEA_tab_15_inv.xls")
inv_raw = inv_raw.drop([0,1,2,3])
inv_raw.columns = inv_raw.loc[4,:]
inv_raw = inv_raw.drop([4, 5])


inv_raw = inv_raw.loc[list_to_keep]
inv_raw =inv_raw.reset_index()
inv_raw = inv_raw.drop(['index'], axis=1)
inv_raw.rename(columns={np.nan:'type'}, inplace=True)

#reformat and create pct changes
inv_raw = inv_raw.T
inv_raw.columns = inv_raw.loc['type',:]
inv_raw = inv_raw.drop(['Line', 'type'], axis = 0)
inv_data_hat =inv_raw.div(inv_raw.shift(1))
inv_data_hat.columns = ['Fixed_assets_and_cons_durables', 'Fixed_assets', 'Private', 'Government', 'Consumer_durables']




In [4]:
#US GDP data in dollars
#note that i have predicted values for durables and services directly in xls file for 2000 and 2001
#I did this for cells c:10 - d:13
GDP_raw = pd.read_excel("Data/BEA_tab_116_GDP_USD.xls")
GDP_raw = GDP_raw.drop([0,1,2,3])
GDP_raw.columns = GDP_raw.loc[4,:]
GDP_raw = GDP_raw.drop([4, 5])
GDP_raw
list_to_keep = [6, 9, 11, 12, 27]
GDP_raw = GDP_raw.loc[list_to_keep]

GDP_raw =GDP_raw.reset_index()
GDP_raw = GDP_raw.drop(['index'], axis=1)
GDP_raw.rename(columns={np.nan:'type'}, inplace=True)

GDP_raw = GDP_raw.T
GDP_raw.columns = GDP_raw.loc['type',:]
GDP_raw = GDP_raw.drop(['Line', 'type'], axis = 0)
GDP_data_hat =GDP_raw.div(GDP_raw.shift(1))

GDP_data_hat.head()
GDP_data_hat.columns = ["GDP", "Durables", "Services", "Gross_private_domestic_investment", "Govt_cons_exp_and_gross_inv"]

In [5]:
#load investment efficiency in changes
chi_data_hat = pd.read_excel("Data/inv_eff_EKNR.xlsx")
chi_data_hat_old = chi_data_hat

#load trade cost shocks
d_data_hat = pd.read_excel("Data/d_EKNR.xlsx")

A_data_hat = pd.read_excel("Data/productivity_EKNR.xlsx")
A_data_hat = A_data_hat.drop([0])
A_data_hat.columns = ['Year', 'country_1', 'country_2']
#read in pi_ijnt
pi_ijt_obs = pd.read_csv("Data/pi_ijt.csv")
pi_ijt_obs = pi_ijt_obs.drop(['Unnamed: 0'], axis=1)
# k_raw = k_raw.drop(['index'], axis=1)

In [6]:
#reindex both
# new_index = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
#             '2013', '2014', '2015', '2016', '2017', '2018']
k_data_hat = k_data_hat.reset_index()
GDP_data_hat = GDP_data_hat.reset_index()

In [7]:
k_data_hat.columns = ['Years', 'Fixed_assets_and_cons_durables',
                         'Fixed_assets',                        'Private',
                           'Government',              'Consumer_durables']
GDP_data_hat.columns = ['Years', 'GDP',
                                'Durables',
                                'Services',
       'Gross_private_domestic_investment',
             'Govt_cons_exp_and_gross_inv']
k_data_hat_old = k_data_hat #for use in function

In [8]:
#decide what you want to use
#using total GDP for Y in calculating factors, and consumer durables only for K
# using consumer durables only for inv. spending
#use only durable GDP for Y when simulating data for beta l and betak

In [9]:
###OPTION 1: construct betaL, betaK myself
#do this for annual data
#make the y, l, m, k data here (all for durables only)
#run in stata and find beta_L_orig, beta_K_orig
#repeat below

##Log inputs (quantity index)
int_inputs_raw = pd.read_excel("Data/BEA_quantity_int_inp_durables.xls")
list_to_keep = [4,5]
int_inputs_raw = int_inputs_raw.loc[list_to_keep,:]
int_inputs_raw.columns = int_inputs_raw.loc[4, :]
int_inputs_raw = int_inputs_raw.drop([4])
int_inputs_raw = int_inputs_raw.drop(['Line', np.nan], axis=1)
int_inputs_raw = int_inputs_raw.T
int_inputs_raw['Years'] = int_inputs_raw.index
int_inputs_raw.columns = ['m', 'Years']
int_inputs_raw = int_inputs_raw.reset_index()
int_inputs_raw = int_inputs_raw.drop([4], axis = 1)
int_inputs_log =np.log(int_inputs_raw['m'])
int_inputs_log = pd.DataFrame(int_inputs_log)
int_inputs_log
int_inputs_log['Years']= int_inputs_raw.Years
int_inputs_log.columns = ['m_log', 'Years']

# ask jonas about units here! does it matter?
##log Durables - GDP (billions)
log_y_durables_data = pd.Series(np.log(GDP_raw.iloc[:,1].values.astype(float)))

##log labor (labor is in 1000s)
labor_raw = pd.read_excel("Data/FRED_emp.xls")
labor_log = pd.Series(np.log(labor_raw['Emp_thousands']))

##log K (billions)
log_K_durables_data = pd.Series(np.log(k_raw.iloc[:,4].values.astype(float)))

lists_for_prod = [pd.DataFrame(int_inputs_log['m_log']),
                  pd.DataFrame(log_y_durables_data),
                  pd.DataFrame(labor_log),
                 pd.DataFrame(log_K_durables_data)]
prod_df = pd.concat(lists_for_prod, axis=1)
prod_df.columns = ['m_log', 'y_log', 'l_log', 'K_log']
prod_df.to_csv('Data/prod_data_US.csv')

In [10]:
###OPTION 1A: construct betaL, betaK myself
#do this for specific categories of consumer durables
#make the y, l, m, k data here (all for durables only)
#run in stata and find beta_L_orig, beta_K_orig
#repeat below
int_inputs_detail_raw = pd.read_excel("Data/detail_prod_data/BEA_m_detail.xls")
list_to_keep = [4,5,6,7,8,9,10,11,12,13,14,15,16]
int_inputs_detail_raw = int_inputs_detail_raw.loc[list_to_keep,:]
int_inputs_detail_raw.columns = int_inputs_detail_raw.loc[4, :]
int_inputs_detail_raw = int_inputs_detail_raw.drop([4])
int_inputs_detail_raw = int_inputs_detail_raw.drop(['Line'], axis=1)
int_inputs_detail_raw.columns = ['category', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
             '2013', '2014', '2015', '2016', '2017', '2018']
int_inputs_detail_raw = int_inputs_detail_raw.reset_index()
int_inputs_detail_raw = int_inputs_detail_raw.drop(['index'], axis=1)
int_inputs_detail_raw = int_inputs_detail_raw.T
int_inputs_detail_raw.columns = int_inputs_detail_raw.iloc[0,:]
int_inputs_detail_raw = int_inputs_detail_raw.drop(['category'])

GDP_detail_raw = pd.read_excel("Data/detail_prod_data/BEA_GDP_detail.xls")
list_to_keep = [4,5,6,7,8,9,10,11,12,13,14,15,16]
GDP_detail_raw = GDP_detail_raw.loc[list_to_keep,:]
GDP_detail_raw.columns = GDP_detail_raw.loc[4, :]
GDP_detail_raw = GDP_detail_raw.drop([4])
GDP_detail_raw = GDP_detail_raw.drop(['Line'], axis=1)
GDP_detail_raw.columns = ['category', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
             '2013', '2014', '2015', '2016', '2017', '2018']
GDP_detail_raw = GDP_detail_raw.reset_index()
GDP_detail_raw = GDP_detail_raw.drop(['index'], axis=1)
GDP_detail_raw = GDP_detail_raw.T
GDP_detail_raw.columns = GDP_detail_raw.iloc[0,:]
GDP_detail_raw = GDP_detail_raw.drop(['category'])
GDP_detail_raw.columns = int_inputs_detail_raw.columns

Labor_detail_raw = pd.read_excel("Data/detail_prod_data/BEA_labor_detail.xls")
list_to_keep = [4,5,6,7,8,9,10,11,12,13,14,15,16, 17, 18]
Labor_detail_raw = Labor_detail_raw.loc[list_to_keep,:]
Labor_detail_raw.columns = Labor_detail_raw.loc[4, :]
Labor_detail_raw = Labor_detail_raw.drop([4,5,6])
Labor_detail_raw = Labor_detail_raw.drop(['Line'], axis=1)
Labor_detail_raw.rename(columns={np.nan:'category'}, inplace=True)
Labor_detail_raw = Labor_detail_raw.T
Labor_detail_raw = Labor_detail_raw.reset_index()
Labor_detail_raw.columns = Labor_detail_raw.iloc[0,:]
Labor_detail_raw = Labor_detail_raw.drop([0])
Labor_detail_raw.index = Labor_detail_raw.category
Labor_detail_raw = Labor_detail_raw.drop(['category'], axis=1)
Labor_detail_raw.columns = int_inputs_detail_raw.columns

#note this raw file is in millions, whereas before it was in billions
#had to windsorize this due to "less treasury stock" from FRED and unavailability of matching categories
K_detail_raw = pd.read_excel("Data/detail_prod_data/FRED_k_Detail_windsorized_new.xlsx")
K_detail_raw.index = K_detail_raw.Years
K_detail_raw = K_detail_raw.drop(['Years'], axis=1)
K_detail_raw.columns = Labor_detail_raw.columns
K_detail_raw = K_detail_raw/1000

In [11]:
# drop year 2000 from labor, GDP, int_inputs because K is lacking this year
Labor_detail_raw = Labor_detail_raw.drop([2000.0])
int_inputs_detail_raw = int_inputs_detail_raw.drop(['2000'])
GDP_detail_raw = GDP_detail_raw.drop(['2000'])


In [12]:
#now take logs and merge into a df
GDP_detail_log = pd.DataFrame(np.log(GDP_detail_raw.values.astype(float)))
GDP_detail_log.columns = Labor_detail_raw.columns

int_inputs_detail_log = pd.DataFrame(np.log(int_inputs_detail_raw.values.astype(float)))
int_inputs_detail_log.columns = Labor_detail_raw.columns

K_detail_log =  pd.DataFrame(np.log(K_detail_raw.values.astype(float)))
K_detail_log.columns = Labor_detail_raw.columns

Labor_detail_log = pd.DataFrame(np.log(Labor_detail_raw.values.astype(float)))
Labor_detail_log.columns = Labor_detail_raw.columns
# np.log(GDP_raw.iloc[:,1].values.astype(float)

In [13]:
#append everything by column type
list_columns = Labor_detail_raw.columns
prod_df_ver_1a = []
index = 0
for i in list_columns:
    index +=1
    index_str = str(index)
    lists_for_prod_ver_1a = [pd.DataFrame(int_inputs_detail_log[i]),
                    pd.DataFrame(GDP_detail_log[i]),
                        pd.DataFrame(Labor_detail_log[i]),
                        pd.DataFrame(K_detail_log[i])]
    df_to_add = pd.concat(lists_for_prod_ver_1a, axis=1)
    df_to_add.columns = ['m_log', 'y_log', 'l_log', 'K_log']
    prod_df_ver_1a.append(df_to_add)
#     df_to_add.to_csv('Data/prod_data_ver_1a/type' + index_str+'.csv')
    df_to_add.to_excel('Data/prod_data_ver_1a/type' + index_str+'.xlsx')

In [14]:
#now write everything as functions tht take beta as inputs

In [15]:
def calc_b_hat(beta_L_for_calc, beta_K_for_calc):
    #now construct b_hat
    ##For EKNR formulation, assume betal = 2/3, betak = 1/3,
    ##For my formulation, betal =  .6670052732854, betak = .2572608033009
    #b_hat = (Yhat/Lhat)^beta_L * (Yhat/Khat)^beta_K
    L_hat = 1 #constant
    b_hat = (GDP_data_hat.GDP/L_hat)**beta_L_for_calc * (GDP_data_hat.GDP/k_data_hat.Consumer_durables)**beta_K_for_calc
    return (b_hat)
    #notice b_hat is 2000, 2001, ...

In [16]:
#now construct p_hat_D
#choose 2 countries of interest
# for now, US and China
# these are indices 
country_full = [
  "Australia",
  "Austria",
  # "Belgium",
  # "Bulgaria",
  "Brazil",
  "Canada",
  "Switzerland",
  "China",
  # "Cyprus",
  # "Czechia",
  "Germany",
  # "Denmark",
  "Spain",
  # "Estonia",
  "Finland",
  "France",
  "United Kingdom",
  "Greece",
  # "Croatia",
  # "Hungary",
  "Indonesia",
  "India",
  # "Ireland",
  "Italy",
  "Japan",
  "Korea, Republic of",
  # "Lithuania",
  # "Luxembourg",
  # "Latvia",
  "Mexico",
  # "Malta",
  "Netherlands",
  "Norway",
  "Poland",
  "Portugal",
  "Romania",
  "Russian Federation",
  "Slovakia",
  "Slovenia",
  "Sweden",
  "Turkey",
  # "Taiwan, Province of China",
  "United States",
  "ROW"
]
c1_ind = country_full.index('United States') #28
c2_ind = country_full.index('China') #5

In [17]:
#get pi data
pi_subset  = pi_ijt_obs[(pi_ijt_obs['Country1'] == c1_ind) | (pi_ijt_obs['Country2'] == c1_ind)]
pi_subset = pi_subset[(pi_subset['Country1'] == c2_ind) | (pi_subset['Country2'] == c2_ind)]
pi_ii_subset  = pi_ijt_obs[(pi_ijt_obs['Country1'] == c1_ind) & (pi_ijt_obs['Country2'] == c1_ind)]
pi_jj_subset  = pi_ijt_obs[(pi_ijt_obs['Country1'] == c2_ind) & (pi_ijt_obs['Country2'] == c2_ind)]

frames = [pi_subset, pi_ii_subset, pi_jj_subset]
pi_subset_full = pd.concat(frames)

In [18]:
pi_subset_full =pi_subset_full.sort_values(['Country1', 'Country2', 'Period'])

In [19]:
# we actually just need pi_US_US (28 28) and pi_US_China (28 5)
pi_for_calc12 = pi_subset_full[(pi_subset_full['Country1']==c1_ind) & (pi_subset_full['Country2']==c2_ind)]
frames = [pi_ii_subset, pi_for_calc12]
pi_for_calc = pd.concat(frames)
pi_for_calc12 = pi_for_calc12.reset_index()
pi_for_calc12.head()
pi_ii_subset = pi_ii_subset.reset_index()

In [20]:
#now subset d_hat in same way as pi
d_data_subset = d_data_hat.drop(['d21', 'd22'],axis=1)
d_data_subset = d_data_subset.drop([0])

In [21]:
#now calculate p_hat_d
def calc_p_hat(beta_L_for_calc, beta_K_for_calc):
    theta_eknr = 2
    p_D_hat = []
    for i in range(0, 18):
        pi_term_1 = pi_for_calc12.pi_value[i] #pi is 2000, 2001, etc.
        pi_term_2 = pi_ii_subset.pi_value[i]
        b_hat = calc_b_hat(beta_L_for_calc, beta_K_for_calc)
        b_hat_term = b_hat[i+1] #bhat is 2000, 2001, etc.
        d_hat_term_1 = d_data_subset.d12[i+1] #2000, 2001
        d_hat_term_2 = d_data_subset.d11[i+1] #2000, 2001
        A_hat_term = A_data_hat.country_1[i+1] #2000, 2001 

        term_1 = pi_term_1 * ((b_hat_term*d_hat_term_1/A_hat_term))**(-1*theta_eknr)
        term_2 = pi_term_2 * ((b_hat_term*d_hat_term_2/A_hat_term))**(-1*theta_eknr)
        pd_term_temp = (term_1 + term_2)**(-1/theta_eknr)
        p_D_hat.append(pd_term_temp)
        
    p_D_hat_df = pd.DataFrame(p_D_hat)
    p_D_hat_df['Year'] = ['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
                '2013', '2014', '2015', '2016', '2017', '2018']
    p_D_hat_df.columns = ['p_D_hat', 'Year']
    return(p_D_hat_df)


In [22]:
def calc_X_pK_hat(beta_L_for_calc, beta_K_for_calc):
    #Create the X/(pK) element, call it x_pK
    X_pK_hat = []
    for i in range(0, 18):
        X_D_term_temp = inv_data_hat.Consumer_durables[i+1]#2000, 2001
        p_D_hat_df = calc_p_hat(beta_L_for_calc, beta_K_for_calc)
        p_D_term_temp = p_D_hat_df.p_D_hat[i]#2001
        K_term_temp =k_data_hat.Consumer_durables[i+1]#2000
        X_pK_temp = X_D_term_temp/(p_D_term_temp * K_term_temp)
        X_pK_hat.append(X_pK_temp)

    X_pK_hat_df = pd.DataFrame(X_pK_hat)
    X_pK_hat_df['Year'] = ['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
                '2013', '2014', '2015', '2016', '2017', '2018']
    X_pK_hat_df.columns = ['X_pK_hat', 'Year']
    return(X_pK_hat_df)

In [23]:
# #merge everything i need into one dataframe
#X_pK, chi, K
# chi_data_hat = chi_data_hat.reset_index()
# k_data_hat = k_data_hat.reset_index()
# X_pK_hat_df = X_pK_hat_df.reset_index()

def get_data(beta_L_for_calc, beta_K_for_calc, GDP_data):
    #calculate Y_hat from GDP_data (only use durables)
    GDP_raw.columns = ['GDP', 'Durable_goods', 'Services', 'G_private_dom_inv', 'Govt_cons_exp_and_gross_inv']
    GDP_subset = GDP_raw['Durable_goods']

    GDP_subset =pd.DataFrame(GDP_subset)
    GDP_subset =GDP_subset.reset_index()
    GDP_subset.columns = ['Year', 'Y']
    GDP_subset['Y_hat'] = GDP_subset.Y.div(GDP_subset.Y.shift(1))

    GDP_subset = GDP_subset.drop([0])
    GDP_subset = GDP_subset.reset_index()

    k_data_hat = k_data_hat_old.drop([0])
    chi_data_hat = chi_data_hat_old.drop([0, 1])
    k_data_hat = k_data_hat.reset_index()
    chi_data_hat = chi_data_hat.reset_index()
    X_pK_hat_df = calc_X_pK_hat(beta_L_for_calc, beta_K_for_calc)
    X_pK_hat_df = X_pK_hat_df.reset_index()
    lists_for_K_LOM = [pd.DataFrame(chi_data_hat['Country 1']),
                      pd.DataFrame(k_data_hat['Consumer_durables']),
                      pd.DataFrame(X_pK_hat_df['X_pK_hat']),
                      pd.DataFrame(GDP_subset['Y_hat'])]
    K_LOM_df = pd.concat(lists_for_K_LOM, axis=1)


    K_LOM_df['Year']= ['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
                '2013', '2014', '2015', '2016', '2017', '2018']
    K_LOM_df.columns = ['chi_hat', 'K_hat', 'X_pK_hat', 'Y_hat', 'Year']
    return (K_LOM_df)

In [24]:
year_data_mom= ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
            '2013', '2014', '2015', '2016', '2017', '2018']


In [31]:
# NEW MOMENTS
# Define data moments - the capital in a given year
from scipy import stats as sts
def calc_data_moments(K_LOM_vals): #doesn't seem to need an argumentb but try for now
#     data_moments = []
    subset_no_recess = K_LOM_vals[(K_LOM_vals.Year.astype(int) <=2007)| (K_LOM_vals.Year.astype(int)>=2013)]
    K_LOM_vals['K_hat_shift'] =K_LOM_vals.K_hat.shift(1) #for m3


    #create a datamoment for each year, this is K_hat_t+2
    data_m_1 = np.mean(subset_no_recess.K_hat) #avg K hat
    data_m_2 = np.mean(subset_no_recess.K_hat/subset_no_recess.Y_hat)#Avg Khat/Y hat
    data_m_3 = sts.pearsonr(K_LOM_vals.K_hat[1:len(K_LOM_vals)], K_LOM_vals.K_hat_shift.dropna())[0]#corr Kt+1, Kt
#     data_m_4 = sum(subset_no_recess.K_hat >=1.03)/len(subset_no_recess.K_hat) #percentage above 1.03(around the mean)

    data_moments = [data_m_1, data_m_2, data_m_3]#, data_m_4]
    return data_moments


def calc_model_moments(chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta):
    
    ##Compute K_hat_t+2
    K_hat_path = []
    #for very first year
    K_hat_path.append(chi_hat[0] * (X_pK_hat[0]**alpha) * (K_hat[0] - (1-delta)) + (1-delta))
    for i in range(1, 18):
        K_t_2 = chi_hat[i] * (X_pK_hat[i]**alpha) * (K_hat_path[i-1] - (1-delta)) + (1-delta)
        K_hat_path.append(K_t_2)
    
    subset_no_recess_K = K_hat_path[0:6] + K_hat_path[11:len(K_hat_path)] # a list
#     print("percentile",np.percentile(subset_no_recess_K, .5))
    subset_no_recess_Y = np.concatenate((Y_hat[0:6],Y_hat[11:len(Y_hat)]))#these are data is that OK?
#     print
    # 0 = 2002
    # 5 = 2007
    # 11 = 2013
    model_m_1 = np.mean(subset_no_recess_K)
    model_m_2 = np.mean(subset_no_recess_K/subset_no_recess_Y)
    model_m_3 = sts.pearsonr(K_hat_path[0:(len(K_hat_path)-1)], K_hat_path[1:len(K_hat_path)])[0] #is this just 1? print this
#     model_m_4 = sum(pd.Series(subset_no_recess_K) >=1.03)/len(subset_no_recess_K)

    model_moments = [model_m_1, model_m_2, model_m_3]#, model_m_4]
    return model_moments
    

In [32]:
#Define the error vector 

def err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta, simple = True):
    data_moments = np.asarray(calc_data_moments(K_LOM_vals)) # 4 0's
    model_moments = np.asarray(calc_model_moments(chi_hat, K_hat, X_pK_hat, Y_hat, alpha,delta))
    
    if simple:
        err_vec = model_moments - data_moments
    else:
        err_vec = (model_moments - data_moments) / data_moments
    return err_vec

In [33]:
#Define criterion function
def criterion(params, *args):
    alpha, delta = params
    K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, W = args
    err = err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta, simple=True)
    crit_val = err.T @ W @ err
    return crit_val

In [34]:
#options
beta_L_EKNR = 2/3
beta_K_EKNR = 1-beta_L_EKNR
beta_L_gmm = .6720710743684
beta_K_gmm = .3279289256316

K_LOM_df_EKNR = get_data(beta_L_EKNR, beta_K_EKNR, GDP_raw)
K_LOM_df_mine = get_data(beta_L_gmm, beta_K_gmm, GDP_raw)

In [35]:
import scipy.optimize as opt
##results when EKNR estimates of betas are used
#Estimate through GMM
# #initial guesses
alpha_init = .5
delta_init=.5

params_init = np.array([alpha_init, delta_init])
W_hat = np.eye(3) 

gmm_args = (K_LOM_df_EKNR, K_LOM_df_EKNR.chi_hat.values, K_LOM_df_EKNR.K_hat.values, K_LOM_df_EKNR.X_pK_hat.values, K_LOM_df_EKNR.Y_hat.values, W_hat)#1 is arbitrary, don't really need this arg
results = opt.minimize(criterion, params_init, args=(gmm_args), tol=1e-14,
                       method='L-BFGS-B',
                       bounds=((1e-10, 1+(1e-10)), (1e-10, 1+(1e-10))))
alpha_gmm, delta_gmm = results.x
print('alpha_hat=', alpha_gmm, 'delta_hat=', delta_gmm)
results

alpha_hat= 0.47544915941430477 delta_hat= 0.1224845905934092


      fun: 0.011678769286144681
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-5.6378513e-08,  0.0000000e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 45
      nit: 13
   status: 0
  success: True
        x: array([0.47544916, 0.12248459])

In [36]:
##results when my estimates of betas are used
#initial guesses
# alpha_init=best[0]
# delta_init=best[1] #given

params_init = np.array([alpha_init, delta_init])
W_hat = np.eye(3) 

gmm_args = (K_LOM_df_mine, K_LOM_df_mine.chi_hat.values, K_LOM_df_mine.K_hat.values, K_LOM_df_mine.X_pK_hat.values, K_LOM_df_mine.Y_hat.values, W_hat)#1 is arbitrary, don't really need this arg
results = opt.minimize(criterion, params_init, args=(gmm_args), tol=1e-14,
                       method='L-BFGS-B',
                       bounds=((1e-10, 1+(1e-10)), (1e-10, 1+(1e-10))))
alpha_gmm, delta_gmm = results.x
print('alpha_hat=', alpha_gmm, 'delta_hat=', delta_gmm)
results

alpha_hat= 0.4746537617161641 delta_hat= 0.12207918631588566


      fun: 0.011698356650983809
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-5.72458747e-08,  5.42968448e-08])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 96
      nit: 14
   status: 0
  success: True
        x: array([0.47465376, 0.12207919])

# END HERE

In [ ]:
# #nontarget moments
# #Define data moments - the capital in a given year
# def calc_data_moments_nontarget(K_LOM_vals): #doesn't seem to need an argumentb but try for now
#     data_moments = []
#     subset_no_recess = K_LOM_vals[(K_LOM_vals.Year.astype(int) <=2007)| (K_LOM_vals.Year.astype(int)>=2013)]


#     data_m_4 = sum(subset_no_recess.K_hat >=1.03)/len(subset_no_recess.K_hat) #percentage above 1.03(around the mean)

#     data_moments.append(data_m_4)
#     #create a datamoment for each year, this is K_hat_t+2
# #     for i in range(0,18):
# #         data_moments.append(K_LOM_vals.K_hat[i])
#     # create variance data moment
#     data_moments.append(np.var(subset_no_recess.K_hat))
#     return data_moments

# def calc_model_moments_nontarget(chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta):
#     model_moments2=[]
#     ##Compute K_hat_t+2
#     K_hat_path = []
#     #for very first year
#     K_hat_path.append(chi_hat[0] * (X_pK_hat[0]**alpha) * (K_hat[0] - (1-delta)) + (1-delta))
#     for i in range(1, 18):
#         K_t_2 = chi_hat[i] * (X_pK_hat[i]**alpha) * (K_hat_path[i-1] - (1-delta)) + (1-delta)
#         K_hat_path.append(K_t_2)
    
#     subset_no_recess_K = K_hat_path[0:6] + K_hat_path[11:len(K_hat_path)] # a list
#     model_m_4 = sum(pd.Series(subset_no_recess_K) >=1.03)/len(subset_no_recess_K)
#     model_moments2.append(model_m_4)

#     # create variance data moment
#     model_moments2.append(np.var(subset_no_recess_K))
    
#     return model_moments2
    

In [ ]:
# data_nontarget = calc_data_moments_nontarget(K_LOM_df_mine)
# model_nontarget =  calc_model_moments_nontarget(K_LOM_df_mine.chi_hat.values, K_LOM_df_mine.K_hat.values, K_LOM_df_mine.X_pK_hat.values, K_LOM_df_mine.Y_hat.values, alpha_gmm, delta_gmm)
# diff_m4 = data_nontarget[0] - model_nontarget[0]
# diff_var = data_nontarget[1] - model_nontarget[1]
# diff_m4
# diff_var

In [ ]:
#compute SEs
#he did a 2-sided numerical derivative, which is the expression above "5. examples"
def Jac_err2(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta, simple = False):
    '''
    This function computes the Jacobian matrix of partial derivatives of the R x 1 moment
    error vector e(x|theta) with respect to the K parameters theta_i in the K x 1 parameter vector
    theta. The resulting matrix is R x K Jacobian.
    '''
    R = 3
    K = 2
    Jac_err = np.zeros((R, K))
    h_alpha = 1e-8 * alpha
    print(h_alpha)
    h_delta = 1e-8 * delta
    
    # def err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta, simple = True):


    
    alpha_upper = alpha +h_alpha
    alpha_lower = alpha -h_alpha
    Jac_err[:, 0] = \
        ((err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha_upper,  delta, simple) -
          err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha_lower, delta, simple)) / (2 * (1e-8)*(alpha))).flatten()
    #.flatten: the jac-err is a 2x2, a slice that is a whole row turns it into a 1d object, so we need to
    #fill it with a 1d object.
    Jac_err[:, 1] = \
        ((err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta + h_delta, simple) -
          err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat,  Y_hat, alpha, delta - h_delta, simple)) / (2 * h_delta)).flatten()
    
    return Jac_err

In [ ]:
import numpy.linalg as lin #good for inverse

N = len(K_LOM_df_mine)
d_err2 = Jac_err2(K_LOM_df_mine, K_LOM_df_mine.chi_hat.values, K_LOM_df_mine.K_hat.values, K_LOM_df_mine.X_pK_hat.values,K_LOM_df_mine.Y_hat.values, alpha_gmm, delta_gmm, True)
# print(d_err2)
SigHat2 = (1 / N) * lin.inv(d_err2.T @ W_hat @ d_err2)
# print(SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. delta_hat=', np.sqrt(SigHat2[1, 1]))
#this leadsto huge standard errors. When i do this with W_hat2, it is much lower

In [ ]:
#the version using the second weighting matrix does not converge!

In [ ]:
d_err2

In [ ]:
#Define criterion function
def criterion2(params, args):
    alpha, delta = params
    K_LOM_vals, chi_hat, K_hat, X_pK_hat, W = args
    err = err_vec(K_LOM_vals, chi_hat, K_hat, X_pK_hat, alpha, delta, simple=True)
    crit_val = err.T @ W @ err
    return crit_val

In [ ]:
# import numpy.linalg as lin


# #perform a grid search, as initial values not so good
# ###Again, assume alpha and delta are parameters of interest (denoted a and d here). Similar intuition, except
# ###uses ParameterGrid instead of nested loops over values of alpha and delta
# from sklearn.model_selection import ParameterGrid
# def obj(params):
#     a = params[0]
#     d = params[1]
#     params_0 = np.array([a, d])
#     gmm_args_0 = (K_LOM_df_mine, K_LOM_df_mine.chi_hat.values, K_LOM_df_mine.K_hat.values, K_LOM_df_mine.X_pK_hat.values, W_hat2)#1 is arbitrary, don't really need this arg


#     lik = criterion2(params_0, args=(gmm_args_0))
#     return -lik
# # Grid search to pick start location
# param_grid = {'alpha': np.arange(0, 1, .01), 'delta': np.arange(0, 1, .01)}
# grid = ParameterGrid(param_grid)
# best = [0, 0]
# best_obj = -1e10
# for params in grid:
#     a = params["alpha"]
#     d = params["delta"]
#     params_0 = np.array([a, d])
#     gmm_args_0 = np.array([K_LOM_df, K_LOM_df.chi_hat.values, K_LOM_df.K_hat.values, K_LOM_df.X_pK_hat.values, W_hat2])#1 is arbitrary, don't really need this arg



#     lik = criterion2(params_0, gmm_args_0)
#     if lik > best_obj:
#         best_obj = lik
#         best = [a, d]



In [ ]:
#repeat with different W
import numpy.linalg as lin


def get_err_mat(K_LOM_vals, chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta, simple = True):
    R = 3 #number of moments
    S = 18 # of obs
    Err_mat = np.zeros((R, S))
    model_points = calc_model_moments(chi_hat, K_hat, X_pK_hat, Y_hat, alpha, delta) #42 values
    data_points = calc_data_moments(K_LOM_vals) #42 values
    if simple:
        for i in range(0, 3): #for i in range (0, 42) #number of moments
            Err_mat[i, :] = data_points[i] - model_points[i]
#             print(data_points[i])
#             print(model_points[i])
    else:
        for i in range(0, 3): #for i in range (0, 42)
            Err_mat[i, :] = (data_points[i] - model_points[i])/model_points[i]
            
    return Err_mat

Err_mat = get_err_mat(K_LOM_df_mine, K_LOM_df_mine.chi_hat.values, K_LOM_df_mine.K_hat.values, K_LOM_df_mine.X_pK_hat.values, K_LOM_df_mine.Y_hat.values, alpha_gmm, delta_gmm,False)
VCV2 = (1 / len(K_LOM_df_mine)) * (Err_mat @ Err_mat.T)
W_hat2 = lin.inv(VCV2)

In [ ]:
Err_mat

In [ ]:
#initial guesses
# alpha_init2 = best[0]
# delta_init2 = best[1]
alpha_init2=.4
delta_init2=.09 #given

params_init2 = np.array([alpha_init2, delta_init2])
gmm_args2 = (K_LOM_df_mine, K_LOM_df_mine.chi_hat.values, K_LOM_df_mine.K_hat.values, K_LOM_df_mine.X_pK_hat.values, K_LOM_df_mine.Y_hat.values, W_hat2)#1 is arbitrary, don't really need this arg

results = opt.minimize(criterion, params_init2, args=(gmm_args2), tol=1e-18,
                       method='L-BFGS-B',
                       bounds=((1e-10, 1-(1e-10)), (1e-10, 1-(1e-10))), options={'ftol': -1e-40, 'gtol': 1e-20})
alpha_gmm2, delta_gmm2 = results.x
print('alpha_hat2=', alpha_gmm2, 'delta_hat2=', delta_gmm2)
results


